In [1]:
import os
import subprocess
import pandas as pd
import numpy as np
import random
import cv2
import matplotlib.pyplot as plt
import re
import pickle 
from tqdm import tqdm
from deepface.modules import modeling, detection, preprocessing

In [3]:
dataset = os.listdir('../../edusense/devemo/Dataset')
print(dataset)

['32', '.DS_Store', '6', '28', '1', '10', '39', '48continue', '4', '3', '2', '13', '5']


In [4]:
def convert_int(s):
    if s.isdigit():
        return int(s)
    else:
        return s


def alphanum_key(s):
    return [convert_int(c) for c in re.split('([0-9]+)', s)]


def sort_nicely(l):
    l.sort(key=alphanum_key)

In [7]:
# 2 lists  to store the following
# list of features (images)
#{0: [[3, 1, 2, 2][4, 3, 2, 2]]}
# list of labels (confused or not)
#{0: 3}

features_list = []
labels_list = []
outfile_name = 'Polish_Dataset.pkl'
# loop over the train and test and validation
frame_count = []
count = 0
for name in dataset:
    # print(ttv, os.path.exists('DataSet/'+ttv+'/'))
    
    if name != '.DS_Store' and os.path.exists('../../edusense/devemo/Dataset/'+name+'/'):
        print(50*'-')
        print(count)
        print('above is index for frame_count')
        print('below is the folder name')
        print(name)
        count += 1

        # train, test, validation
        
        # return all images in this directory
        content_list = os.listdir('../../edusense/devemo/Dataset/'+name+'/')
        # remove video for below
        img_list = [c for c in content_list if c.endswith('.jpg')]
        sort_nicely(img_list)
        frame_count.append(len(img_list)) 


--------------------------------------------------
0
above is index for frame_count
below is the folder name
32
--------------------------------------------------
1
above is index for frame_count
below is the folder name
6
--------------------------------------------------
2
above is index for frame_count
below is the folder name
28
--------------------------------------------------
3
above is index for frame_count
below is the folder name
1
--------------------------------------------------
4
above is index for frame_count
below is the folder name
10
--------------------------------------------------
5
above is index for frame_count
below is the folder name
39
--------------------------------------------------
6
above is index for frame_count
below is the folder name
48continue
--------------------------------------------------
7
above is index for frame_count
below is the folder name
4
--------------------------------------------------
8
above is index for frame_count
below is the fo

In [8]:
index_grouping = [[0, 2], [1], [3, 9], [4], [5], [6], [7], [8], [9], [10], [11]]

In [11]:
[frame_count[i[1]] + frame_count[i[0]] for i in index_grouping if len(i) > 1]

[1536, 4470]

In [12]:
import statistics

In [13]:
# Combine sums for len(i) > 1 and len(i) == 1
combined_sums = [
    sum(frame_count[idx] for idx in i) for i in index_grouping
]

# Calculate the median
median_value = statistics.median(combined_sums)

In [14]:
median_value

539

In [15]:
np.percentile(frame_count, [10, 90])

array([ 140.6, 1248.1])

In [8]:
# 2 lists  to store the following
# list of features (images)
#{0: [[3, 1, 2, 2][4, 3, 2, 2]]}
# list of labels (confused or not)
#{0: 3}


# loop over the train and test and validation
for ttv in dataset:
    # print(ttv, os.path.exists('DataSet/'+ttv+'/'))
    
    if ttv != '.DS_Store' and os.path.exists('../DAiSEE/DataSet/'+ttv+'/'):
        # train, test, validation
        count = 0
        features_list = []
        labels_list = []
        
        users = os.listdir('../DAiSEE/DataSet/'+ttv+'/')
        # print(users)
        if ttv == 'Test':
            all_clips = test_csv[['ClipID', 'Confusion']]
            outfile_name = 'Test.pkl'
            # print(all_clips)
        if ttv == 'Train':
            all_clips = train_csv[['ClipID', 'Confusion']]
            outfile_name = 'Train.pkl'
        if ttv == 'Validation':
            all_clips = valid_csv[['ClipID', 'Confusion']]
            outfile_name = 'Validation.pkl'
            
        for user in tqdm(users):
            if user != '.DS_Store':
                # user is the folder, has 300 jpeg
                # extract jpeg
                currUser = os.listdir('../DAiSEE/DataSet/'+ttv+'/'+user+'/')
                
                # extract is still a folder (10s clip each folder)
                for extract in currUser:
                    
                    if extract != '.DS_Store':
                        content_list = os.listdir('../DAiSEE/DataSet/'+ttv+'/'+user+'/'+extract+'/')
                                        
                        if pd.Series(content_list).isin(all_clips['ClipID']).any():
                            confusion_val = all_clips.loc[all_clips['ClipID'].isin(content_list)].Confusion.to_list()[0]
                            # remove video for below
                            img_list = [c for c in content_list if c.endswith('.jpg')]
                            sort_nicely(img_list)
                            # if confusion_val == 0 or 1, skip every 30 (1s)
                            # elif confusion_val == 2 or 3, skip every 3 (0.1s)
                            if confusion_val == 0: #6000
                                chosen = img_list[149] 
                            elif confusion_val == 1: #2200 * 1 = 2200
                                chosen = img_list[149] 
                            elif confusion_val == 2: # 750 * 11 = 8250
                                chosen = img_list[::27]
                            elif confusion_val == 3: # 101 * 80 = 8080
                                chosen = img_list[::4]
                            # chosen  = random.choice(img_list)
                            for item in img_list: 
                                
                                if item in chosen:
                                    count += 1
                                    img_path = os.path.abspath('.')+'/../DAiSEE/DataSet/'+ttv+'/'+user+'/'+extract+'/'+item
                                    img = cv2.imread(str(img_path))
                                    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                                    img_gray = cv2.resize(img_gray, (128, 128))
                                    
                                    # img is 3d array (w, h, 1), label is confusion_val
                                    features_list.append(img_gray)
                                    # if confusion_val is 0 or 1, then = 0
                                    if confusion_val == 0 or confusion_val == 1:
                                        labels_list.append(0)
                                    # # elif confusion_val is 2 or 3, then = 1 and 2
                                    elif confusion_val == 2 or confusion_val == 3:
                                        labels_list.append(confusion_val-1)
                                    # labels_list.append(confusion_val)
        
        with open(outfile_name, 'wb') as f:
            pickle.dump([features_list, labels_list], f)
                                        
                        # clip = os.listdir('DataSet/'+ttv+'/'+user+'/'+extract+'/')[0]
                        # print (clip[:-4])
                        # path = os.path.abspath('.')+'/DataSet/'+ttv+'/'+user+'/'+extract+'/'
                        # split_video(clip, clip[:-4], path)
        # print(ttv, count)
# print ("================================================================================\n")
# print ("Frame Extraction Successful")

100%|██████████| 22/22 [00:05<00:00,  4.27it/s]
